<a href="https://colab.research.google.com/github/fabiobasson/my-repo/blob/main/oficios_qualidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! rm -rf camelot

In [ ]:
#!git clone https://www.github.com/camelot-dev/camelot
#!cd camelot
!pip install "camelot-py[cv]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.5/249.5 kB 19.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of camelot-py[cv] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.2 MB/s eta 0:00:00


In [ ]:
!apt-get install ghostscript

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 libgs9 libgs9-common
  libidn11 libijs-0.35 libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript libgs9
  libgs9-common libidn11 libijs-0.35 libjbig2dec0 poppler-data
0 upgraded, 10 newly installed, 0 to remove and 34 not upgraded.
Need to get 12.7 MB of archives.
After this operation, 51.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 fonts-droid-fallback all 1:6.0.1r16-1.1 [1,805 kB]
Get:2 h

In [ ]:
!pip install 'PyPDF2<3.0'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: PyPDF2
    Found existing installation: PyPDF2 3.0.1
    Uninstalling PyPDF2-3.0.1:
      Successfully uninstalled PyPDF2-3.0.1


In [ ]:
!pip install xlwt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 6.3 MB/s eta 0:00:00


In [26]:
import camelot
tables = camelot.read_pdf('Laudo_2628256_7_BUZ_33_RJS_22111210.pdf')

In [27]:
tables.n

1

In [28]:
print(tables[0].parsing_report)
>>> {'accuracy': 100.0, 'whitespace': 0.0, 'order': 1, 'page': 1}
for i in range(tables.n):
    print(tables[i].parsing_report)
>>> {'accuracy': 100.0, 'whitespace': 0.0, 'order': 1, 'page': 1}
>>> {'accuracy': 100.0, 'whitespace': 0.0, 'order': 2, 'page': 1}

{'accuracy': 100.0, 'whitespace': 37.5, 'order': 1, 'page': 1}
{'accuracy': 100.0, 'whitespace': 37.5, 'order': 1, 'page': 1}


{'accuracy': 100.0, 'whitespace': 0.0, 'order': 2, 'page': 1}

In [29]:
print(tables[0])

for i in range(tables.n):
    print(tables[i])


<Table shape=(14, 4)>
<Table shape=(14, 4)>


In [30]:
tables[0].to_excel('oficios_qualidade.xls')


/usr/local/lib/python3.10/dist-packages/camelot/core.py:614: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  writer = pd.ExcelWriter(path)
/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/camelot/core.py:616: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [35]:
tables[0].df[[1,0,2]]

,1,0,2
0,,3. RELAÇÃO DE DADOS RECEBIDOS,
1,Norma de Referência:,Tipos de Dados:,Situação dos Dados no BDEP*:
2,RANP n° 880/2022,3.1. Arquivos Auxiliares - Esquemas de Ferrame...,NÃO CONFORME
3,RANP n° 880/2022,3.2. Arquivos Auxiliares - Dados Direcionais,APROVADO
4,RANP n° 880/2022,3.3. Arquivos Auxiliares - Relatório de Campo ...,
5,RANP n° 880/2022,3.4. Perfis Durante a Perfuração,NÃO CONFORME
6,RANP n° 880/2022,3.5. Perfis Convencionais,APROVADO
7,RANP n° 880/2022,3.6. Perfis Especiais - Teste de Formação a Cabo,APROVADO
8,RANP n° 880/2022,3.7. Perfis Especiais - Sísmica de Poço,
9,RANP n° 880/2022,3.8. Perfis Especiais - Amostrador Lateral,APROVADO


In [36]:
tables[0].df

,0,1,2,3
0,3. RELAÇÃO DE DADOS RECEBIDOS,,,
1,Tipos de Dados:,Norma de Referência:,Situação dos Dados no BDEP*:,
2,3.1. Arquivos Auxiliares - Esquemas de Ferrame...,RANP n° 880/2022,NÃO CONFORME,
3,3.2. Arquivos Auxiliares - Dados Direcionais,RANP n° 880/2022,APROVADO,
4,3.3. Arquivos Auxiliares - Relatório de Campo ...,RANP n° 880/2022,,
5,3.4. Perfis Durante a Perfuração,RANP n° 880/2022,NÃO CONFORME,
6,3.5. Perfis Convencionais,RANP n° 880/2022,APROVADO,
7,3.6. Perfis Especiais - Teste de Formação a Cabo,RANP n° 880/2022,APROVADO,
8,3.7. Perfis Especiais - Sísmica de Poço,RANP n° 880/2022,,
9,3.8. Perfis Especiais - Amostrador Lateral,RANP n° 880/2022,APROVADO,


In [39]:
!pip install pdfplumber -q
!pip install os -q

import os, pdfplumber

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [86]:
Arquivos= os.listdir()

pdf = pdfplumber.open('Laudo_2628256_7_BUZ_33_RJS_22111210.pdf')

pdf.pages



[<Page:1>, <Page:2>, <Page:3>, <Page:4>]

In [87]:
page = pdf.pages[0]
page

<Page:1>

In [95]:
text = page.extract_text()
text 


'Agencia Nacional de Petróleo, Gás Natura e Biocombustíveis\nSuperintendência de Dados Técnicos\nBanco de Dados de Exploração e Produção\nLaudo de Avaliação de Dados de Poço\n1. CABEÇALHO\n1.1. Nº do Laudo 1.2. Data da 1.3. Nome do 1.4. Nº Cadastro do 1.5. Conclusão do 1.6. Empresa\nde Reprovação: Avaliação: Poço: Poço: Poço: Operadora:\n2211.1.0210 18/11/2022 7-BUZ-33-RJS 74316029460 13/03/2020 Petrobras\n2. CONTROLE DE REMESSAS (USO INTERNO)\n2.1. Número de Protocolo da(s) 2.2. Data de Protocolo da(s) 2.3. Código de Cadastro da(s)\nRemessa(s): Remessa(s): Mídia(s) no BDEP:\n48610.207606/2020-74 22/05/2020 301_2020\n48610.206434/2020-11 30/04/2020 254_2020\n3. RELAÇÃO DE DADOS RECEBIDOS\nTipos de Dados: Norma de Referência: Situação dos Dados no BDEP*:\n3.1. Arquivos Auxiliares - Esquemas de Ferramentas RANP n° 880/2022 NÃO CONFORME\n3.2. Arquivos Auxiliares - Dados Direcionais RANP n° 880/2022 APROVADO\n3.3. Arquivos Auxiliares - Relatório de Campo de Sísmica RANP n° 880/2022\n3.4. P

In [116]:
text.split('\n')[5]

'1.1. Nº do Laudo 1.2. Data da 1.3. Nome do 1.4. Nº Cadastro do 1.5. Conclusão do 1.6. Empresa'

In [113]:
text.split('\n')[7].split('\t')

['2211.1.0210 18/11/2022 7-BUZ-33-RJS 74316029460 13/03/2020 Petrobras']